In [ ]:
# librerias

import numpy as np
import cv2
import matplotlib.pyplot as plt
import pandas as pd


In [ ]:
# parametros

rect = 150

# crea el dataframe

drosophila = pd.DataFrame(columns=['id', 'pixels' ,'x', 'y', 'sex', 'state', 'targetX', 'targetY'])

font = cv2.FONT_HERSHEY_SIMPLEX

In [ ]:

# incrementa el brillo de la imagen

def increase_brightness(img, value = 30):
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    h, s, v = cv2.split(hsv)

    lim = 255 - value
    v[v > lim] = 255
    v[v <= lim] += value

    final_hsv = cv2.merge((h, s, v))
    img = cv2.cvtColor(final_hsv, cv2.COLOR_HSV2BGR)
    return img

In [ ]:
# binarizacion, br cantidad de brillo, gb = kernel del gaussian blur, li y ls limites superior e inferior del theadhold

def image_threshold(img, br = 150, gb = 3, li = 230, ls= 255):
    img_b = increase_brightness(img, value=br)
    img_b  = cv2.cvtColor(img_b, cv2.COLOR_BGR2GRAY)
    img_b = cv2.GaussianBlur(img_b, (gb,gb), 0)
    _, img_c = cv2.threshold(img_b, li, ls, cv2.THRESH_BINARY)
    return img_c

In [ ]:
# Morphological Operations: erosiona y dilata ... k tamano del kernel, d_iter, e_iter iteraciones en dilatacion y erosion

def image_erodil(img, d_iter = 10, e_iter = 10, k = 3):
    # erossion and dilatation
    kernel = np.ones((k,k), np.uint8)

    img = cv2.dilate(img, kernel, iterations = d_iter)
    img = cv2.erode(img, kernel, iterations = e_iter)
    return  img


In [ ]:
def image_overlap(img, rect= 150, show = False):
    
        temp_cont, _ = cv2.findContours(img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        ret = 'sin clasificar'

        if (len(temp_cont)>1):                      # cuenta contornos en temp_img
            
            ret = 'overlaped'

            if show:                                # imprime superposición
                plt.figure(figsize = (100,10))
                plt.imshow(img)
            
        return ret
    

In [ ]:
import random

def img_classification(img):
    sex = random.choice(["f", "m"])
    return sex




In [ ]:

def image_addBoundingbox(img, x0, y0, ov, text, q, y_text = 10, s = "n/a"):
    c = (0,0,0)
    if ov == "sin clasificar": c = (0,128,0)        # verde
    if ov == "overlaped": c = (255,0,0)             # rojo
    if ov == "fuera umbral 1": c = (0,0,0)          # negro
    if ov == "fuera umbral 2": c = (255,255,0)        # marron
    if ov == "classified": 
        if s == "f": c = (255,0,255)      # rosa
        if s == "m": c = (0,0,255)        # azul
    cv2.rectangle(img, (x0, y0), (x0+rect, y0+rect), c, 5)

    if text:
        cv2.putText(img, str(q), (x0, y0-y_text), font, 1, c, 3, cv2.LINE_AA)


    
    return img


In [ ]:
# detecta y analiza contornos, t? rect: tamanno del recuadro, pMin, pMax: % maximo y minimo de pixes ocupados

def image_contours(img_t, img_o, drosophila, writePNG = False, t = 10, rect = 150, pMin = 4, pMax = 20):
    img_t = cv2.bitwise_not(img_t)
    contornos, _ = cv2.findContours(img_t, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    img_fin = img_o.copy()

    hmin = 20
    wmin = 20
    hmax = 120
    wmax = 120
    y_text = 10

    cont = 0

    for c in contornos:
        if len(c)>t:                                # que carajos es t??? y este proc???

            (x, y, w, h) = cv2.boundingRect(c)                          # realiza el bounding box

            x0 = int((x + w/2) - (rect/2))                              # localiza la coordenada x0 basado en el tamaño del recuadro
            y0 = int((y + h/2) - (rect/2))                              # localiza la coordenada y0 basado en el tamaño del recuadro
            
            cont = cont + 1                                             # incrementa la cantidad de objetos encontrados
            
            if ((h>hmin) and (w>wmin)) and ((h<hmax) and (w<wmax)):     # tamaño minimo y maximo del recuadro

                roi1 = img_t[y0:y0+rect, x0:x0+rect]                    # roi de imagen binarizada
                q = cv2.countNonZero(roi1)                              # calculo de % de pixels ocupados
                p = round((q/(rect*rect))*100, 1)
                
                if (p>pMin) and (p<pMax):                               # seleccion de % minimo y maximo

                    ov = image_overlap(roi1)                            # verifica superposición de imagenes
                
                    if writePNG:
                        roi = img_o[y0:y0+rect, x0:x0+rect] 
                        cv2.imwrite(str(cont)+'.png', roi)
                    
                else:

                    ov = "fuera umbral 1"
                
            else:

                ov = "fuera umbral 2"
            

            img_fin = image_addBoundingbox(img_fin, x0, y0, ov, True, cont, y_text)

            drosophila.loc[len(drosophila.index)] = [cont, p, x0, y0, "n/a", ov, 0, 0]   # registra el roi


    return img_fin    

In [ ]:
# marca el centro de masa, cross tamaño de la mira

def roi_target(roi, img, x0, y0, cross = 30):
    
    c = int(cross/2)                         #  la cruz tiene 2*cross de lado
    
    eyes_lo=np.array([90,60,60])
    eyes_hi=np.array([125,85,85])

    mask=cv2.inRange(roi, eyes_lo, eyes_hi)
    M = cv2.moments(mask)
 

    # calculate x,y coordinate of center
    x = int(M["m10"] / M["m00"])
    y = int(M["m01"] / M["m00"])
 
    x = x0 + x
    y = y0 + y

    # put text and highlight the center
    cv2.circle(img, (x, y), 5, (255, 255, 255), -1)
    cv2.putText(img, "target", (x - 25, y - 25), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
 
    

    cv2.line(img, (x, y-c), (x, y+c), (255, 255, 0), 2) 
    cv2.line(img, (x-c, y), (x+c, y), (255, 255, 0), 2) 

    return x, y, img
    


In [ ]:
# marca el centro de masa, cross tamaño de la mira (teniendo en cuenta que son boundingboxes con errores)

def roi_target_plus(roi, img, x0, y0, cross = 30):

    c = int(cross/2)                         #  la cruz tiene 2*cross de lado

    roi = cv2.bitwise_not(roi)

    kernel = np.ones((3,3), np.uint8)
    roi = cv2.erode(roi, kernel, iterations = 10)
    contornos, _ = cv2.findContours(roi, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)


    for contor in contornos:
        (x, y, w, h) = cv2.boundingRect(contor) 
        x = int(x + w/2) + x0
        y = int(y + h/2) + y0

        # put text and highlight the center
        cv2.circle(img, (x, y), 5, (255, 255, 255), -1)
        cv2.putText(img, "target", (x - 25, y - 25), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
 
        cv2.line(img, (x, y-c), (x, y+c), (255, 255, 0), 2) 
        cv2.line(img, (x-c, y), (x+c, y), (255, 255, 0), 2) 

    return x, y, img
    

In [ ]:
def image_classification(img, img_de, drosophila, rect = 150):

    # recorre el dataset de las que estan en estado "sin clasificar"

    img_f = img.copy()

    for _, row in drosophila.iterrows():

        targetX = 0
        targetY = 0

        x0 = int(row.x)
        y0 = int(row.y)

        roi = img[y0:y0+rect, x0:x0+rect]  
        st = row.state

        s = 'n/a'
        if (row.state == 'sin clasificar'):
            s = img_classification(roi)                                                 # clasifica la mosca
            st = 'classified'
            

        if (s=="f") and ((x0>0) and (y0>0)):
            targetX, targetY, img_f = roi_target(roi, img, x0, y0, 30)                  # encuentra punto de target de moscas ok

        if (st != "classified") and ((x0>0) and (y0>0)):
            roi = img_de[y0:y0+rect, x0:x0+rect]  
            targetX, targetY, img_f = roi_target_plus(roi, img, x0, y0, 30)                  # encuentra punto de target de moscas con problemas

        # actualiza el dataframe
        drosophila.loc[drosophila['id'] == row.id, ['state']] = st
        drosophila.loc[drosophila['id'] == row.id, ['sex']] = s
        drosophila.loc[drosophila['id'] == row.id, ['targetX']] = targetX
        drosophila.loc[drosophila['id'] == row.id, ['targetY']] = targetY
        

        # redibuja boundingboxes
        #if ((s=="f") or (s=="m")): 
        img_f = image_addBoundingbox(img_f, x0, y0, st, True, row.id, 10, s)

    return img_f

In [ ]:
img_orig = cv2.imread('images/Fotos 1-11-097.jpeg', cv2.COLOR_BGR2RGB) 
img_orig = cv2.cvtColor(img_orig, cv2.COLOR_RGB2BGR)

img_pre = image_threshold(img_orig, 150, 3, 230, 255)
img_de = image_erodil(img_pre, 10, 10, 3)

plt.figure(figsize = (200,20))
plt.imshow(img_de) 

img_c = image_contours(img_de, img_orig, drosophila, False)

plt.figure(figsize = (200,20))
plt.imshow(img_c) 

# clasifica, genera el target y el recorrido del laser en las que estan verde (sin clasificar)

img_f = image_classification(img_orig, img_de, drosophila, rect = 150)

plt.figure(figsize = (200,20))
plt.imshow(img_f)

In [ ]:
drosophila.head()


In [ ]:
drosophila.tail()

In [ ]:


laser_optRoute()






    x0 = drosophila.at[_, "x"] 
    y0 = drosophila.at[_, "y"] 
    temp_img = img_erodil[y0:y0+rect, x0:x0+rect]
    temp_cont, _ = cv2.findContours(temp_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # cuenta contornos en temp_img

    if (len(temp_cont)>1):
        
        


        print(row.id)

        # imprime superposición
        plt.figure(figsize = (100,10))
        plt.imshow(temp_img)
        cv2.rectangle(img_fin2, (x0, y0), (x0+rect, y0+rect), (255, 0, 0), 6)


                    
